### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

ratings_min_count = 100

ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings['dateTime'] = pd.to_datetime(ratings['timestamp'], unit='s')

ratings_2 = ratings.groupby('userId').agg({'dateTime': ['min', 'max'], 'rating': 'count'})
ratings_2['lifespan'] = (ratings_2['dateTime']['max'] - ratings_2['dateTime']['min']).astype('timedelta64[h]')

mean_life_span = ratings_2[ratings_2['rating']['count']>ratings_min_count]['lifespan'].mean()

print(f"Среднее время жизни: {mean_life_span:.2f} часов")

Среднее время жизни: 11132.94 часов


### Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [2]:
import pandas as pd

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

revenue = (client_base[['client_id']]
           .merge(rzd, on= 'client_id', how = 'left')
           .merge(auto, on= 'client_id', how = 'left')
           .merge(air, on= 'client_id', how = 'left')
           .fillna(0)
          )
print("Выручка без указания адреса")
print(revenue)

print("")
revenue_address = (revenue
           .merge(client_base, on= 'client_id', how = 'left')
           .fillna("")
          )
print("Выручка с указанием адреса")
print(revenue_address)

Выручка без указания адреса
   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           0.0          0.0
1        112       2810.0           0.0          0.0
2        113      10283.0       57483.0          0.0
3        114       5774.0          83.0          0.0
4        115        981.0         912.0         81.0
5        116          0.0        4834.0          4.0
6        117          0.0          98.0         13.0
7        118          0.0           0.0        173.0

Выручка с указанием адреса
   client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           0.0          0.0  Комсомольская 4
1        112       2810.0           0.0          0.0   Энтузиастов 8а
2        113      10283.0       57483.0          0.0  Левобережная 1а
3        114       5774.0          83.0          0.0          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1
5        116          0.0        4834.0        

### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Как добавить данные?
- Я предполагаю, что данные о геолокации должны быть связаны непосредственно с информацией о каждой сессии/визите
- В таблице покупок должна быть ссылка на сессию или пользователя/сессию
- Думаю целесообразно выполнить препроцессинг координат и добавлять в сессию/покупку ссылки на географический объект (например, город). Таким образом можно сформировать структуру для более общего анализа (например, легко объединить города в регионы/страны и т.д.)
- Таким образом, таблица с данными по координатам может выглядеть так: пользователь / сессия / [широта / долгота или гео.объект] / [другие метрики сессии]

Как повлияет удаление точек друг от друга?
- Если вести работу с "сырыми" координатами, то будет очень сложно проводить группировку и вообще анализ по этой метрике, т.к. даже небольшое расстояние для физического объекта может означать новый набор координат
- С другой стороны с числовыми данными можно проводить более детальный анализ: кластерный анализ, расстояние между точками входа и т.д.

Дополнительные признаки, сведения
1. Район, город, регион, страна, часть света (это позволит, как минимум, снизить размерность таблицы данных о визитах)
2. Группировка или кластеризация точек входа для пользователя или группы пользователей (это позволит, например, выявить регионы где приложение пользуется большей популярностью)
3. Анализ точек входа и точек входа с покупкой (например следюущая гипотеза, в центре города пользователи выбирают товары, но совершают покупки в спальных районах)
